In [2]:
# Mount Google Drive to Colab
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!pwd
#!ls -al
%cd '/content/gdrive/My Drive/NLP/Part2'
#!ls -al

/content
/content/gdrive/My Drive/NLP/Part2


# 作業 : 觀察機器翻譯 ATTENTION 內容 
- 仔細地觀察機器翻譯 ATTENTION 結果

# [作業目標]
- 透過視覺化 注意力 attention 層 了解attention 的作用方式

# [作業重點]
- 透過視覺化 注意力 attention 層 了解attention 的作用方式
- 原則上只要之前的訓練有跑完，這邊的程式可以執行成功最後只要觀察結果就好


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

#from torchtext.data import Field, BucketIterator, TabularDataset
from torchtext.legacy.data import Field, BucketIterator, TabularDataset
from sklearn.model_selection import train_test_split
import csv

import numpy as np
import re
import random
import math
import time

import csv
import spacy

In [7]:
!mkdir ./data
!mkdir ./data/multi30k
!python -m spacy download en
!ls ./data/multi30k -al
spacy_english = spacy.load("en_core_web_sm")
!ls ./data/multi30k -al

mkdir: cannot create directory ‘./data’: File exists
mkdir: cannot create directory ‘./data/multi30k’: File exists
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
total 5448
-rw------- 1 root root   66154 Apr  7 09:05 mmt_task1_test2016.tar.gz
-rw------- 1 root root   70649 Oct 17  2016 test2016.de
-rw------- 1 root root   62076 Oct 17  2016 test2016.en
-rw------- 1 root root   72261 Feb 11  2017 test2016.fr
-rw------- 1 root root 2110399 Feb  2  2016 train.de
-rw------- 1 root root 1801239 Feb  2  2016 train.en
-rw------- 1 root root 1207136 Apr  7 09:05 training.tar.gz
-rw------- 1 root root   75920 Feb  2  2016 val.de
-rw------- 1 root root   63298 Feb  2  2016 val.en
-rw------- 1 root root   46329 Apr  7 09:05 validation.tar.gz
total 5448
-rw------- 1 roo

In [8]:
ls -al /usr/local/lib/python3.7/dist-packages/matplotlib/mpl-data/fonts/ttf

total 6836
drwxr-sr-x 2 root staff   4096 Apr  7 13:48 ./
drwxr-sr-x 5 root staff   4096 Apr  7 13:48 ../
-rw-r--r-- 1 root staff  25680 Apr  7 13:40 cmb10.ttf
-rw-r--r-- 1 root staff  21092 Apr  7 13:40 cmex10.ttf
-rw-r--r-- 1 root staff  32560 Apr  7 13:40 cmmi10.ttf
-rw-r--r-- 1 root staff  26348 Apr  7 13:40 cmr10.ttf
-rw-r--r-- 1 root staff  20376 Apr  7 13:40 cmss10.ttf
-rw-r--r-- 1 root staff  29396 Apr  7 13:40 cmsy10.ttf
-rw-r--r-- 1 root staff  28136 Apr  7 13:40 cmtt10.ttf
-rw-r--r-- 1 root staff 641720 Apr  7 13:40 DejaVuSans-BoldOblique.ttf
-rw-r--r-- 1 root staff 704128 Apr  7 13:40 DejaVuSans-Bold.ttf
-rw-r--r-- 1 root staff  25712 Apr  7 13:40 DejaVuSansDisplay.ttf
-rw-r--r-- 1 root staff 253116 Apr  7 13:40 DejaVuSansMono-BoldOblique.ttf
-rw-r--r-- 1 root staff 331536 Apr  7 13:40 DejaVuSansMono-Bold.ttf
-rw-r--r-- 1 root staff 251472 Apr  7 13:40 DejaVuSansMono-Oblique.ttf
-rw-r--r-- 1 root staff 340240 Apr  7 13:40 DejaVuSansMono.ttf
-rw-r--r-- 1 root staff 633840 Ap

In [10]:
9# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載字體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O taipei_sans_tc_beta.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download
!mv taipei_sans_tc_beta.ttf /usr/local/lib/python3.7/dist-packages/matplotlib//mpl-data/fonts/ttf

from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt 
plt.style.use("seaborn-whitegrid")
import matplotlib.ticker as ticker
# 自定義字體變數
myfont = FontProperties(fname=r'/usr/local/lib/python3.7/dist-packages/matplotlib/mpl-data/fonts/ttf/taipei_sans_tc_beta.ttf')

--2021-04-12 05:29:16--  https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.google.com (drive.google.com)... 64.233.189.138, 64.233.189.100, 64.233.189.113, ...
Connecting to drive.google.com (drive.google.com)|64.233.189.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ptu6vs7vs58oct21eh36temllurqlr9j/1618205325000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_ [following]
--2021-04-12 05:29:18--  https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ptu6vs7vs58oct21eh36temllurqlr9j/1618205325000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving doc-0k-9o-docs.googleusercontent.com (doc-0k-9o-docs.googleusercontent.com)... 108.177.97.132, 2404:6800:4008:c00::84
Connecting to doc-0k-9o-docs.googleusercontent.com (doc-0k-9o-docs.googleusercontent.com)|108.17

In [38]:
data_dir = '/content/drive/My Drive/NLP/Part2/data/'
lines = open(data_dir + 'cmn.txt' , encoding='utf-8').read().strip().split('\n')
trnslt_pairs = [[s for s in l.split('\t')] for l in lines ]
print ("Sample: " , trnslt_pairs[1000][0:2] )
print ("Total records:" , len(trnslt_pairs))

Sample:  ['He was drowned.', '他被淹死了。']
Total records: 24360


In [40]:
# 下載 spacy 的英文模型 幫我們做tokenize
model_dir =  '/content/drive/My Drive/NLP/Part2/model/'

spacy_eng = spacy.load('en_core_web_sm')
def tokenize_eng(text):
  #清除不需要的字符
  text = re.sub(r"([.!?])", r" \1", text)
  return [tok.text for tok in spacy_eng.tokenizer(text)]

TRG = Field(tokenize = tokenize_eng, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)



def tokenize_cmn(text):
  #去掉非中文字元
  regex = re.compile(r'[^\u4e00-\u9fa5A-Za-z0-9]')
  text = regex.sub(' ', text)

  return [word for word in text if word.strip()]
    

SRC = Field(tokenize = tokenize_cmn, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            include_lengths = True)




train_dataset, dev_dataset, test_dataset = TabularDataset.splits(
    path = data_dir , format = 'csv', skip_header = True,
    train='train.csv', validation='val.csv', test='test.csv',
    fields=[
        ('trg', TRG),
        ('src', SRC)
    ]
)


# 讀取之前儲存的 vocabulary
SRC.vocab = torch.load(model_dir + 'SRC_vocab.pt')
TRG.vocab = torch.load(model_dir + 'TRG_vocab.pt')

print ("中文語料的字元表長度: " , len(SRC.vocab) , ", 英文的字元表長度: " ,len(TRG.vocab))
print ("Sample SRC:", test_dataset[0].src , "TRG:", test_dataset[0].trg)

BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, dev_dataset, test_dataset), 
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.src),
     device = device)

中文語料的字元表長度:  2696 , 英文的字元表長度:  4105
Sample SRC: ['你', '选', '好', '专', '业', '了', '吗'] TRG: ['have', 'you', 'chosen', 'a', 'major', 'yet', '?']


# 模型主體 和前面範例程式一樣



In [26]:
class Attention(nn.Module):
  def __init__(self, enc_hid_dim, dec_hid_dim):
    super().__init__()

  def forward(self, hidden, encoder_outputs, mask):
    # hidden bz , dec_hid_dim
    # encoder_outputs src len, bz , enc_hid_dim x 2
    # mask bz , src len
    
    batch_size = encoder_outputs.shape[1]
    src_len = encoder_outputs.shape[0]

    hidden = hidden.unsqueeze(1) 
    # hidden unsqueeze bz , 1 , dec_hid_dim

    attention = torch.matmul( hidden , encoder_outputs.permute(1, 2, 0)   )
    # attention bz, 1 , src len
    
    attention = attention.squeeze(1)
    # squeeze bz , src len

    attention = attention.masked_fill(mask == 0, -1e10)

    return F.softmax(attention, dim = 1)

In [41]:
class RNNEncoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        # 雙向 ＧＲＵ encoder 
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        
        #src shape [src len, batch size]
        #src_len shape [batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded shape [src len, batch size, emb dim]
                
        # 使用pack_padded_sequence 來壓縮序列        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len)
                
        packed_outputs, hidden = self.rnn(packed_embedded)

        # 使用 pad_packed_sequence 用來展開序列成原本形狀的      
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
            
        #outputs shape [src len, batch size, hid dim * num directions]
        #hidden shape [n layers * num directions, batch size, hid dim]
        
        #hidden 堆疊 [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs 是最後一層 
        
        #hidden [-2, :, : ] 是最後一層 forwards RNN 
        #hidden [-1, :, : ] 是最後一層 backwards RNN
        
        # hidden 是最後再過一層 dense layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs shape [src len, batch size, enc hid dim * 2]
        #hidden shape [batch size, dec hid dim]
        
        return outputs, hidden

class RNNDecoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        # 單向 ＧＲＵ decoder 
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):
             
        #input shape [batch size]
        #hidden shape [batch size, dec hid dim]
        #encoder_outputs shape [src len, batch size, enc hid dim * 2]
        #mask shape [batch size, src len]
        
        input = input.unsqueeze(0)
        
        #input shape [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded shape [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs, mask)
                
        #a shape [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a shape [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs shape [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted shape [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted shape [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input shape [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output shape [seq len, batch size, dec hid dim * n directions]
        #hidden shape [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output shape [1, batch size, dec hid dim]
        #hidden shape [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction shape [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

class Seq2SeqATTN(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        mask = self.create_mask(src)

        #mask = [batch size, src len]
                
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
            
        return outputs

# 建立模型和重要參數 請保持和前面訓練時一樣

In [42]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 256 # 注意 encoder hidden layer 設定 必須為 dec 的一半 
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

LEARNING_RATE = 0.002

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = RNNEncoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = RNNDecoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)
model = Seq2SeqATTN(enc, dec, SRC_PAD_IDX, device).to(device)

optimizer = optim.Adam(model.parameters(),lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)


def initial_mdl_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(initial_mdl_weights)
print ("模型全部參數量: {:10,d} ".format(sum(p.numel() for p in model.parameters())))
model

模型全部參數量: 10,020,873 


Seq2SeqATTN(
  (encoder): RNNEncoder(
    (embedding): Embedding(2696, 256)
    (rnn): GRU(256, 256, bidirectional=True)
    (fc): Linear(in_features=512, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): RNNDecoder(
    (attention): Attention()
    (embedding): Embedding(4105, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=4105, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [45]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_len = batch.src
            trg = batch.trg

            output = model(src, src_len.cpu(), trg, 0) #turn off teacher forcing
            
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [46]:
model_dir =  '/content/drive/My Drive/NLP/Part2/model/'
model.load_state_dict(torch.load(model_dir + 'best-model.pt'))
#model.load_state_dict(torch.load(model_dir + 'model-7.pt'))
test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.662 | Test PPL:  38.943 |


In [47]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
        
    #if isinstance(sentence, str):
    #    nlp = spacy_en = spacy.load('en_core_web_sm')
    #    tokens = [token.text.lower() for token in spacy_en(sentence)]
    #else:
    #    tokens = [token.lower() for token in sentence]

    tokens = [token.lower() for token in sentence]
        
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor, src_len.cpu())

    mask = model.create_mask(src_tensor)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)

        attentions[i] = attention
            
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attentions[:len(trg_tokens)-1]

In [48]:
def display_attention(sentence, translation, attention):
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    
    attention = attention.squeeze(1).cpu().detach().numpy()
    
    cax = ax.matshow(attention, cmap='bone')
   
    #fontdict = {"fontproperties": zhfont}
    
    #ax.set_xticks(range(max(max_len_tar, len(predicted_seq))))
    #ax.set_xlim(-0.5, max_len_tar -1.5)
    
    #ax.set_yticks(range(len(sentence) + 2))
    #ax.set_xticklabels([subword_encoder_zh.decode([i]) for i in predicted_seq 
    #                    if i < subword_encoder_zh.vocab_size], 
    #                   fontdict=fontdict, fontsize=18)
    
    #plt.rcParams["font.family"]="sans-serif"
    #plt.rcParams['font.sans-serif']=['STSong'] #用来正常显示中文标签
    
    ax.tick_params(labelsize=15)
    ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                       rotation=45 , fontproperties=myfont) #, fontdict=fontdict)
    ax.set_yticklabels(['']+translation, fontproperties=myfont) # , fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

# 作業重點
## 請選擇一個好的翻譯結果
## 將其 ATTENTION 視覺化 


In [63]:
# 請在這邊自行調整 sample index 
# 觀察不同句子的 ATTENTION 結果
example_idx = 899

src = vars(train_dataset.examples[example_idx])['src']
trg = vars(train_dataset.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

src = ['你', '能', '檢', '查', '一', '下', '這', '個', '輪', '胎', '的', '氣', '壓', '嗎']
trg = ['could', 'you', 'check', 'the', 'tire', 'pressure', '?']
predicted trg = ['could', 'you', 'check', 'this', 'tire', 'tire', '?', '<eos>']


# 請觀察翻譯文 和被翻譯文的語意對應

In [ ]:
print ("".join(src ))
display_attention(src, translation, attention)